In [101]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [103]:
from src.inference import get_feature_store, fetch_predictions
import pandas as pd
from datetime import datetime, timedelta, timezone


def fetch_days_data(days):
    current_date = pd.to_datetime(datetime.now(timezone.utc))
    fetch_data_from = current_date - timedelta(days=(365+days)) 
    fetch_data_to = current_date - timedelta(days=365)
    print(fetch_data_from, fetch_data_to)
    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    # query = query.filter((fg.pickup_hour >= fetch_data_from))
    df = query.read()
    cond = (df["pickup_hour"] >= fetch_data_from) & (df["pickup_hour"] <= fetch_data_to)
    return df[cond]

In [104]:
ts_data = fetch_days_data(180)

2023-09-06 17:26:23.297405+00:00 2024-03-04 17:26:23.297405+00:00
2025-03-04 12:26:23,322 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 12:26:23,356 INFO: Initializing external client
2025-03-04 12:26:23,357 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 12:26:23,915 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.59s) 


In [105]:
from src.data_utils import transform_ts_data_info_features_and_target

features, targets = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [106]:
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()
pipeline.fit(features, targets)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.170658 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171646
[LightGBM] [Info] Number of data points in the train set: 24180, number of used features: 675
[LightGBM] [Info] Start training from score 16.950703


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000019AFED9C4A0>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [107]:
from sklearn.metrics import mean_absolute_error
predictions = pipeline.predict(features)

In [108]:
test_mae = mean_absolute_error(targets, predictions)
print(f"{test_mae:.4f}")

2.9275


In [109]:
from src.inference import load_metrics_from_registry 

metric = load_metrics_from_registry()

2025-03-04 12:27:09,675 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 12:27:09,691 INFO: Initializing external client
2025-03-04 12:27:09,691 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 12:27:10,207 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673


In [110]:
metric

{'test_mae': 1.935489834251996}

In [111]:
from src.inference import load_model_from_registry
model = load_model_from_registry()

2025-03-04 12:27:12,848 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 12:27:12,862 INFO: Initializing external client
2025-03-04 12:27:12,866 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 12:27:13,386 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673


In [112]:
import joblib  
import src.config
# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

['D:\\EAS-500\\sp25_taxi-main\\models\\lgb_model.pkl']

In [113]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(features)
output_schema = Schema(targets)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [114]:
model_registry = project.get_model_registry()
model_registry

ModelRegistry(project: 'EAS500')

In [115]:
modelv2 = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor V2",
    input_example=features.sample(),
    model_schema=model_schema,
)

In [116]:
modelv2.save('D:\\EAS-500\\sp25_taxi-main\\models\\lgb_model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/316176 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/2049 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/51418 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1215673/models/taxi_demand_predictor_next_hour/12


Model(name: 'taxi_demand_predictor_next_hour', version: 12)

In [117]:
from src.inference import load_model_from_registry
model = load_model_from_registry(-1)

2025-03-04 12:27:31,728 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 12:27:31,738 INFO: Initializing external client
2025-03-04 12:27:31,739 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 12:27:32,354 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673


In [118]:
models = model_registry.get_models(name=config.MODEL_NAME)

In [119]:
models

[Model(name: 'taxi_demand_predictor_next_hour', version: 3),
 Model(name: 'taxi_demand_predictor_next_hour', version: 11),
 Model(name: 'taxi_demand_predictor_next_hour', version: 4),
 Model(name: 'taxi_demand_predictor_next_hour', version: 6),
 Model(name: 'taxi_demand_predictor_next_hour', version: 7),
 Model(name: 'taxi_demand_predictor_next_hour', version: 9),
 Model(name: 'taxi_demand_predictor_next_hour', version: 10),
 Model(name: 'taxi_demand_predictor_next_hour', version: 5),
 Model(name: 'taxi_demand_predictor_next_hour', version: 2),
 Model(name: 'taxi_demand_predictor_next_hour', version: 12),
 Model(name: 'taxi_demand_predictor_next_hour', version: 1),
 Model(name: 'taxi_demand_predictor_next_hour', version: 8)]

In [120]:
max(models, key=lambda model: model.version)


Model(name: 'taxi_demand_predictor_next_hour', version: 12)

In [121]:
load_metrics_from_registry()

2025-03-04 12:27:38,798 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 12:27:38,813 INFO: Initializing external client
2025-03-04 12:27:38,813 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 12:27:39,389 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673


{'test_mae': 2.9274555379359004}